This notebook outlines several methods for tokenizing text into words (and sentences), including:

* Code to accompany the chapter "Natural Language Corpus Data"
from the book "Beautiful Data" (Segaran and Hammerbacher, 2009)
http://oreilly.com/catalog/9780596157111/

Code copyright (c) 2008-2009 by Peter Norvig

You are free to use this code under the MIT licencse: 
http://www.opensource.org/licenses/mit-license.php


highlighting differences between them.

In [1]:
import re, string, random, glob, operator, heapq
from collections import defaultdict
from math import log10
from functools import reduce
print ('done')

done


In [2]:
def memo(f):
    "Memoize function f."
    table = {}
    def fmemo(*args):
        if args not in table:
            table[args] = f(*args)
        return table[args]
    fmemo.memo = table
    return fmemo
print ('done')

done


#### Word Segmentation (p. 223)

In [3]:
N = 1024908267229 ## Number of tokens

def datafile(name, sep='\t'):
    "Read key,value pairs from file."
    for line in open(name):
        yield line.split(sep)

def avoid_long_words(key, N):
    "Estimate the probability of an unknown word."
    return 10./(N * 10**len(key))

class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.itervalues()))
        self.missingfn = missingfn or (lambda k, N: 1./N)
    def __call__(self, key): 
        if key in self: return self[key]/self.N  
        else: return self.missingfn(key, self.N)


        
Pw  = Pdist(datafile('data/count_1w.txt'), N, avoid_long_words)

#### segment2: second version, with bigram counts, (p. 226-227)

def cPw(word, prev):
    "Conditional probability of word, given previous word."
    try:
        return P2w[prev + ' ' + word]/float(Pw[prev])
    except KeyError:
        return Pw(word)

P2w = Pdist(datafile('data/count_2w.txt'), N)
print ('done')

done


In [4]:
def splits(text, L=20):
    "Return a list of all possible (first, rem) pairs, len(first)<=L."
    return [(text[:i+1], text[i+1:]) 
            for i in range(min(len(text), L))]

def product(nums):
    "Return the product of a sequence of numbers."
    return reduce(operator.mul, nums, 1)

def Pwords(words): 
    "The Naive Bayes probability of a sequence of words."
    return product(Pw(w) for w in words)
print ('done')

done


In [5]:
@memo
def segment(text):
    "Return a list of words that is the best segmentation of text."
    if not text: return []
    candidates = ([first]+segment(rem) for first,rem in splits(text))
    return max(candidates, key=Pwords)
print ('done')

done


In [6]:
segment('colorlessgreenideassleepfuriously.')

['colorless', 'green', 'ideas', 'sleep', 'furiously', '.']